In [1]:
import numpy as np
import fluxion.comp_graph as cg

import autograd as ag

In [2]:
d_in = 7
d_out = 1
d_hid = 13

B = 17 # batch size
w_A_np = np.random.normal(size=(d_in, d_hid))
w_C_np = np.random.normal(size=(d_hid, d_out))
x_np = np.random.normal(size=(B, d_in))
omega_true = np.random.normal(size=(d_in, 1))
def true_fct(z):
    return np.cos(np.dot(z, omega_true))
y_true = np.array([true_fct(z) for z in x_np])

x = cg.Value("x", x_np) # input vector

w_A = cg.Value("w_A", w_A_np) # params in linear layer
linear1 = cg.Dot("linear1")

act1 = cg.Tanh("act1")

w_C = cg.Value("w_C", w_C_np) # params in a linear layer
linear2 = cg.Dot("linear2")

# loss function comput node
loss = cg.MSELoss("mse")

# we use calls to forward pass to define the function topology
x.forward()
w_A.forward()
w_C.forward()
y = linear1.forward(x, w_A)
y = act1.forward(linear1)
y = linear2.forward(act1, w_C)

# finally we pass inputs through a loss function that 
# returns a scalar
l = loss.forward(linear2, y_true)

In [3]:
def test_embed_function(input):
    z = ag.numpy.dot(input, w_A_np)
    z = ag.numpy.tanh(z)
    z = ag.numpy.dot(z, w_C_np)
    return z

def test_loss_function(input):
    z = test_embed_function(input)
    l = ag.numpy.mean( (z - y_true)**2. , keepdims=True)
    return l

In [4]:
atol = 1e-5

y_ag = test_embed_function(x_np)
l_ag = test_loss_function(x_np)
assert np.allclose(y, y_ag, atol=atol)
assert np.allclose(l, l_ag, atol=atol)

In [5]:
my_graph = cg.Graph("my_fun", [loss])
my_graph.backward()

In [6]:
ag_grad_fn = ag.elementwise_grad(test_loss_function)
dl_dx_ag = ag_grad_fn(x_np)

assert np.allclose(x.d_out, dl_dx_ag, atol=atol)

In [7]:
x.d_out.shape

(17, 7)

In [8]:
dl_dx_ag.shape

(17, 7)

In [9]:
x.d_out

array([[-0.00130078,  0.09370173,  0.02079846, -0.0591128 ,  0.00766673,
         0.05984277,  0.03011341],
       [ 0.06229296,  0.02121124, -0.00147247, -0.1320522 ,  0.02405879,
        -0.02435105, -0.0416249 ],
       [-0.09327006, -0.02826   , -0.0436361 ,  0.04801962, -0.04448927,
         0.01635916, -0.01289019],
       [-0.07183066, -0.03320991, -0.0126541 ,  0.02298268, -0.03067478,
         0.00556124,  0.00826458],
       [ 0.03780593, -0.03089557,  0.01045615,  0.02575978,  0.02048321,
        -0.01954853,  0.01870209],
       [-0.02081396,  0.12589773, -0.0555805 , -0.039016  , -0.05236376,
        -0.02044785, -0.14384473],
       [ 0.18593376, -0.30173307, -0.15808933, -0.10230736,  0.08703317,
         0.10960068,  0.45324985],
       [ 0.03391318, -0.0038191 , -0.00808106, -0.00693938,  0.00488159,
        -0.01738208, -0.02176767],
       [-0.03188743,  0.01845321,  0.00076951, -0.01488566, -0.01556722,
         0.03163639,  0.01570709],
       [ 0.00137754,  0.0697

In [10]:
dl_dx_ag

array([[-0.00130078,  0.09370173,  0.02079846, -0.0591128 ,  0.00766673,
         0.05984277,  0.03011341],
       [ 0.06229296,  0.02121124, -0.00147247, -0.1320522 ,  0.02405879,
        -0.02435105, -0.0416249 ],
       [-0.09327006, -0.02826   , -0.0436361 ,  0.04801962, -0.04448927,
         0.01635916, -0.01289019],
       [-0.07183066, -0.03320991, -0.0126541 ,  0.02298268, -0.03067478,
         0.00556124,  0.00826458],
       [ 0.03780593, -0.03089557,  0.01045615,  0.02575978,  0.02048321,
        -0.01954853,  0.01870209],
       [-0.02081396,  0.12589773, -0.0555805 , -0.039016  , -0.05236376,
        -0.02044785, -0.14384473],
       [ 0.18593376, -0.30173307, -0.15808933, -0.10230736,  0.08703317,
         0.10960068,  0.45324985],
       [ 0.03391318, -0.0038191 , -0.00808106, -0.00693938,  0.00488159,
        -0.01738208, -0.02176767],
       [-0.03188743,  0.01845321,  0.00076951, -0.01488566, -0.01556722,
         0.03163639,  0.01570709],
       [ 0.00137754,  0.0697